<font size="1">Copyright 2021, by the California Institute of Technology. ALL RIGHTS RESERVED. United States Government sponsorship acknowledged. Any commercial use must be negotiated with the Office of Technology Transfer at the California Institute of Technology.</font>
    
<font size="1">This software may be subject to U.S. export control laws and regulations. By accepting this document, the user agrees to comply with all applicable U.S. export laws and regulations. User has the responsibility to obtain export licenses, or other export authority as may be required, before exporting such information to foreign countries or providing access to foreign persons.<font>

# Introduction
In this notebook, you will use RESTful calls to the Pele API to register as a user, the utilize the Pele client library which provides automatic session renewal.

#### Kernel: isce, plant or mintpy

## User Registration

In order to use the Pele API, you must be a registered user. First set some parameters:

In [ ]:
import requests, json, getpass
from requests.auth import HTTPBasicAuth
import urllib3

urllib3.disable_warnings()

# set the base url to interact with the goddess, Pele
base_url = input("Enter Pele REST API base url (e.g. https://<mozart_ip>/pele/api/v0.1) then press <Enter>: ")
print("Using base url {}.".format(base_url))

In [ ]:
# set username
user = input("Enter email address then press <Enter>: ")
print("Using {}.".format(user))

In [ ]:
# set password
print("Enter password then press <Enter>.")
print("*ATTENTION!!! DON'T USE YOUR JPL PASSWORD HERE. MAKE SOMETHING UP FOR THIS TEST.*")
password = getpass.getpass()
print("password: '{}'".format(password))


### Note: If you are not yet registered, the following cell *should* produce a 401 / Unauthorized Access response. This is the correct behavior.

In [ ]:
# login with a username and password
r = requests.post(base_url + '/login', auth=HTTPBasicAuth(user, password), verify=False)

# expect 401 (Unauthorized access) error
print("status code: {}".format(r.status_code))
print("content: {}".format(r.content.decode()))
assert r.status_code == 401

Register with a valid email address:

In [ ]:
r = requests.post(base_url + '/register', data={'email': user, 'password': password}, verify=False)

# expect 201 (created)
print("status code: {}".format(r.status_code))
print("content: {}".format(r.content.decode()))
assert r.status_code == 201

The email you receive will contain a verification code. For example:

```
Use your verification code below to verify your Pele API account at http://localhost:8877/api/v0.1/:

ffa8d18b-f581-44bf-8864-b52a2cd8e7b6
```
You will use that verification code to verify your user account:

In [ ]:
# prompt for verification code
ver_code = input("Enter the verification code:")

# verify
r = requests.post(base_url + '/verify', data={'email': user, 'verification_code': ver_code}, verify=False)

# expect 200
print("status code: {}".format(r.status_code))
print(json.dumps(r.json(), indent=2))
assert r.status_code == 200

## Create the .netrc file for automatic login

If you will be running scripts that will interact with the Pele API, you will need to utilize a method for automatically logging into the Pele REST API to request the API token and to refresh the token should the token expire during the script's execution.

The Pele requests client can utilize the `.netrc` file to automate this for you.

Here we populate your .netrc:

In [ ]:
from urllib.parse import urlparse
import getpass, os, stat

system_username = getpass.getuser()
# parse url to get netloc component
pr = urlparse(base_url)

print("netloc: {}".format(pr.netloc))

# get home directory
stream = os.popen('ls -d ~')
home_dir = stream.read().strip()

# create .netrc
print(f"Writing {home_dir}/.netrc")
with open(f"{home_dir}/.netrc", 'a') as f:
    f.write(f"machine {pr.netloc} login {user} password {password}\nmacdef init\n\n")

# fix perms
os.chmod(f"{home_dir}/.netrc", stat.S_IRUSR | stat.S_IWUSR)

At this point you are ready to use the Pele client API w/ automatic token/session refresh. Feel free to explore the remaining optional portions of this notebook which discusses the API token (which is now transparent to you through the Pele client API) and rate limiting of calls.

## login to get API token

Once your user registration is verified, you can then log into the Pele API which will provide you with a time-bombed API token. You will use this API token to subsequently make requests to the Pele API:

In [ ]:
r = requests.post(base_url + '/login', auth=HTTPBasicAuth(user, password), verify=False)

# expect 200
print("status code: {}".format(r.status_code))
print(r)
print(json.dumps(r.json(), indent=2))

# extract API token
token = r.json()['token']
print("token: {}".format(token))
assert r.status_code == 200

pass your API token in a header called `X-API-KEY`:

In [ ]:
r = requests.get(base_url + '/test/echo', params={'echo_str': 'hello world'}, headers={'X-API-KEY': token}, verify=False)

# expect 200
print("status code: {}".format(r.status_code))
print(json.dumps(r.json(), indent=2))
assert r.status_code == 200

## refresh API token after expiration

As stated before, the API token you receive is time-bombed. It will expire after a certain amount of time (defaults to 86400 seconds). The example below shows that when your token has expired, you will receive a `401` status code error with the error message: `Expired token. Reauthentication required.`:

```
In [1]: r = requests.get(base_url + '/test/echo', params={'echo_str': 'hello world'}, headers={'X-API-KEY': token})

In [2]: r.status_code
Out[2]: 401

In [3]: r.json()
Out[3]: 
{u'authenticated': False,
 u'message': u'Expired token. Reauthentication required.'}
```

At this point, you will have to login again to refresh your API token (see [login to get API token](#login-to-get-API-token) above).

#### Issue a request through the Pele client

In [ ]:
from pele_client.client import PeleRequests

# instantiate PeleRequests object
print(f"Base URL {base_url}")
pr = PeleRequests(base_url, verify=False)

# now use like requests module (`request()`, `get()`, `head()`, `post()`, `put()`, `delete()`, `patch()`)
r = pr.get(base_url + '/test/echo', params={'echo_str': 'hello world'})

# expect 200
print("status code: {}".format(r.status_code))
print(json.dumps(r.json(), indent=2))
assert r.status_code == 200

## Rate-limited API calls

The Pele REST API rate-limits calls to prevent DoS-like access to the backend database:

In [ ]:
for i in range(20):
    r = pr.get(base_url + '/test/echo', params={'echo_str': f'({i}) hello world'})
    print("({}) status code: {}".format(i, r.status_code))
    print(json.dumps(r.json(), indent=2))
    r.raise_for_status()

To mitigate this, utilize exponential backoff:

In [ ]:
!pip install backoff

In [ ]:
import backoff
from requests.exceptions import HTTPError

@backoff.on_exception(backoff.expo, HTTPError, max_tries=5, max_value=5)
def echo(i):
    r = pr.get(base_url + '/test/echo', params={'echo_str': f'({i}) hello world'})
    print("({}) status code: {}".format(i, r.status_code))
    print(json.dumps(r.json(), indent=2))
    r.raise_for_status()
    
for i in range(20):
    echo(i)

<font size="1">This notebook is compatible with NISAR Jupyter Server Stack v1.7.1 and above</font>